In [ ]:
from MPLearn.embedding_notebook import *
from MPLearn.view_cells import *
import mysql.connector
initialize_notebook()

In [ ]:
experiment_path = '../S25'
#too smooth
embedding_tag = "UMAP_viral_plate_scaled_neighbors=5_neg_sampling_rate=50_epochs=1000_a=1.5_b=.2_20201019"
embedding_tag = "UMAP_viral_plate_scaled_neighbors=15_neg_sampling_rate=20_epochs=2000_20201019"

# too small of clusters
embedding_tag = "UMAP_viral_plate_scaled_neighbors=5_neg_sampling_rate=200_epochs=2000_b=.2_20201019"

embedding_tag = "UMAP_viral_plate_scaled_umap_n_neighbors=300_20201019"
embedding_tag = "UMAP_viral_plate_scaled_n_neighbors=30_20201019"
embedding_tag = "UMAP_viral_plate_scaled_n_neighbors=15_20201019"
embedding_tag = "UMAP_viral_plate_scaled_n_neighbors=5_20201019"

embedding_tag = "UMAP_viral_plate_scaled_n_neighbors=5_negative_sample_rate=50_20201019"
embedding_tag = "UMAP_viral_plate_scaled_n_neighbors=30_umap_negative_sample_rate=50_20201019"
embedding_tag = "UMAP_viral_plate_scaled_n_neighbors=30_umap_negative_sample_rate=50_a=2_b=.2_20201019"
embedding_tag = "UMAP_viral_plate_scaled_n_neighbors=30_umap_negative_sample_rate=50_a=2_b=.6_20201019"
embedding_tag = "UMAP_viral_plate_scaled_n_neighbors=30_umap_negative_sample_rate=50_a=20_b=.6_20201019"
embedding_tag = "UMAP_viral_plate_scaled_n_neighbors=30_umap_negative_sample_rate=50_a=50_b=.6_20201019"
embedding_tag = "UMAP_viral_plate_scaled_n_neighbors=30_umap_negative_sample_rate=50_a=50_b=.2_20201019"
embedding_tag = "UMAP_viral_no_zp0_plate_scaled_n_neighbors=30_umap_negative_sample_rate=50_a=50_b=.2_20201019"


meta_path = "../S25/raw_data/viral_plate_scaled_MasterDataTable.parquet"



output_path = '../S25/product/figures/infected_patch_1999B_2020A_2021A_20201017'
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
def load_embedding():
    meta_columns=[
        'plate_id',
        'well_id',
        'field_id',
        'condition',
        'row', 'column',
        'drug_1',
        'drug_1_concentration',
        'drug_1_units',
        'drug_2',
        'drug_2_concentration',
        'drug_2_units',
        'ImageNumber',
        'plate_id',
        'ImageNumber',
        'Number_Object_Number',
        'AreaShape_Center_X',
        'AreaShape_Center_Y']

    embedding = load_single_embedding(
        experiment_path=experiment_path,
        embedding_tag=embedding_tag,
        meta_path=meta_path,
        meta_columns=meta_columns,
        verbose=True)

    return embedding

viral_embedding = load_embedding()

In [ ]:
%output size=300
view_UMAP(viral_embedding[viral_embedding.UMAP_1 < 5][viral_embedding.UMAP_2 < 5])

In [ ]:
%output size=400
view_UMAP_clusters(viral_embedding[viral_embedding.UMAP_1 < 5][viral_embedding.UMAP_2 < 5])

In [ ]:
print("Fixing meta data to locate images...")
viral_embedding['Plate_Name'] = 'SARS_' + viral_embedding.plate_id
viral_embedding.loc[:, 'Image_Metadata_WellID'] = viral_embedding.well_id
viral_embedding.loc[:, 'Image_Metadata_FieldID'] = viral_embedding.field_id
viral_embedding.loc[viral_embedding.plate_id == '1999B', 'Image_Metadata_PlateID'] = "20200427T024129"
viral_embedding.loc[viral_embedding.plate_id == '2020A', 'Image_Metadata_PlateID'] = "20200508T124847"
viral_embedding.loc[viral_embedding.plate_id == '2021A', 'Image_Metadata_PlateID'] = "20200512T002847"
viral_embedding['microscope_plate_id'] = viral_embedding.Image_Metadata_PlateID
viral_embedding['ViralObj_Number_Object_Number'] = viral_embedding.Number_Object_Number
viral_embedding['ViralObj_AreaShape_Center_X'] = viral_embedding.AreaShape_Center_X
viral_embedding['ViralObj_AreaShape_Center_Y'] = viral_embedding.AreaShape_Center_Y


image_config = {
    "database_connection_info" : "/home/ubuntu/.mysql/connectors.cnf",
    "database_options_group" : "covid19cq1",
    "S3_region" : 'us-east-1',
    "S3_bucket" : "sextoncov19",
    "S3_key_template" : "CQ1/{microscope_plate_id}_{plate_id}/Projection/W{Image_Metadata_WellID}F{Image_Metadata_FieldID}T0001Z000C{Dye_Number}.tif",
    "key_object" : "ViralObj",
    "dyes" : ["CMO", "Hoe", "Lipids", "Virus"],
    "saturations" : [0.20, 0.60, 0.15, 0.10],
    "color_maps" : ['Blues', 'Greens', 'viridis', 'inferno'],
    "width" : 400,
    "height" : 400,
    "verbose" : True
}


print("Collating instances...")
from MPLearn.view_cells import collate_cell_instances

collate_cell_instances(
    cell_ids = viral_embedding,
    group_dimension = "cluster_label",
    group_values = [1,4,2,3,5],
    n_instances_per_group = 20,
    output_fname = f"{output_path}/cluster_instances_workbook_20_exampels.xlsx",
    image_config = image_config,
    image_cell_height=1300,
    image_cell_width=57,
    verbose = True)